# Classification

This tutorial uses safeds on **titanic passenger data** to predict who will survive and who will not, using sex as a feature for the prediction.


1. Load your data into a `Table`, the data is available under `docs/tutorials/data/titanic.csv`:


In [1]:
from safeds.data.tabular.containers import Table

titanic = Table.from_csv_file("data/titanic.csv")
#For visualisation purposes we only print out the first 15 rows.
titanic.slice_rows(0, 15)

id,name,sex,age,siblings_spouses,parents_children,ticket,travel_class,fare,cabin,port_embarked,survived
i64,str,str,f64,i64,i64,str,i64,f64,str,str,i64
0,"""Abbing, Mr. Anthony""","""male""",42.0,0,0,"""C.A. 5547""",3,7.55,null,"""Southampton""",0
1,"""Abbott, Master. Eugene Joseph""","""male""",13.0,0,2,"""C.A. 2673""",3,20.25,null,"""Southampton""",0
2,"""Abbott, Mr. Rossmore Edward""","""male""",16.0,1,1,"""C.A. 2673""",3,20.25,null,"""Southampton""",0
3,"""Abbott, Mrs. Stanton (Rosa Hun…","""female""",35.0,1,1,"""C.A. 2673""",3,20.25,null,"""Southampton""",1
4,"""Abelseth, Miss. Karen Marie""","""female""",16.0,0,0,"""348125""",3,7.65,null,"""Southampton""",1
…,…,…,…,…,…,…,…,…,…,…,…
10,"""Adahl, Mr. Mauritz Nils Martin""","""male""",30.0,0,0,"""C 7076""",3,7.25,null,"""Southampton""",0
11,"""Adams, Mr. John""","""male""",26.0,0,0,"""341826""",3,8.05,null,"""Southampton""",0
12,"""Ahlin, Mrs. Johan (Johanna Per…","""female""",40.0,1,0,"""7546""",3,9.475,null,"""Southampton""",0


2. Split the titanic dataset into two tables. A training set, that we will use later to implement a training model to predict the survival of passengers, containing 60% of the data, and a testing set containing the rest of the data.
Delete the column `survived` from the test set, to be able to predict it later:

In [2]:
train_table, testing_table = titanic.split_rows(0.6)

test_table = testing_table.remove_columns(["survived"]).shuffle_rows()

3. Use `OneHotEncoder` to create an encoder, that will be used later to transform the training table.
* We use `OneHotEncoder` to transform non-numerical categorical values into numerical representations with values of zero or one. In this example we will transform the values of the sex column, hence they will be used in the model for predicting the surviving of passengers.
* Use the `fit` function of the `OneHotEncoder` to pass the table and the column names, that will be used as features to predict who will survive to the encoder.
* The names of the column before transformation need to be saved, because `OneHotEncoder` changes the names of the fitted `Column`s:


In [3]:
from safeds.data.tabular.transformation import OneHotEncoder

encoder = OneHotEncoder(column_names="sex").fit(train_table)

4. Transform the training table using the fitted encoder, and create a set with the new names of the fitted `Column`s:


In [4]:
transformed_table = encoder.transform(train_table)

5. Mark the `survived` `Column` as the target variable to be predicted. Include some columns only as extra columns, which are completely ignored by the model:

In [5]:
extra_names = ["id", "name", "ticket", "cabin", "port_embarked", "age", "fare"]

train_tabular_dataset = transformed_table.to_tabular_dataset("survived", extra_names=extra_names)

6. Use `RandomForest` classifier as a model for the classification. Pass the "train_tabular_dataset" table to the fit function of the model:

In [6]:
from safeds.ml.classical.classification import RandomForestClassifier

model = RandomForestClassifier()
fitted_model= model.fit(train_tabular_dataset)

7. Use the fitted random forest model, that we trained on the training dataset to predict the survival rate of passengers in the test dataset.
Transform the test data with `OneHotEncoder` first, to be able to pass it to the predict function, that uses our fitted random forest model for prediction:

In [7]:
transformed_test_table = encoder.transform(test_table)

prediction = fitted_model.predict(
    transformed_test_table
)
#For visualisation purposes we only print out the first 15 rows.
prediction.to_table().slice_rows(start=0, length=15)

id,name,age,siblings_spouses,parents_children,ticket,travel_class,fare,cabin,port_embarked,sex__female,sex__male,survived
i64,str,f64,i64,i64,str,i64,f64,str,str,u8,u8,i64
301,"""Danoff, Mr. Yoto""",27.0,0,0,"""349219""",3,7.8958,null,"""Southampton""",0,1,0
477,"""Greenfield, Mr. William Bertra…",23.0,0,1,"""PC 17759""",1,63.3583,"""D10 D12""","""Cherbourg""",0,1,0
386,"""Farrell, Mr. James""",40.5,0,0,"""367232""",3,7.75,null,"""Queenstown""",0,1,0
864,"""Newell, Miss. Madeleine""",31.0,1,0,"""35273""",1,113.275,"""D36""","""Cherbourg""",1,0,1
267,"""Cor, Mr. Ivan""",27.0,0,0,"""349229""",3,7.8958,null,"""Southampton""",0,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…
1044,"""Ryerson, Master. John Borie""",13.0,2,2,"""PC 17608""",1,262.375,"""B57 B59 B63 B66""","""Cherbourg""",0,1,1
641,"""Keane, Mr. Daniel""",35.0,0,0,"""233734""",2,12.35,null,"""Queenstown""",0,1,0
222,"""Chambers, Mrs. Norman Campbell…",33.0,1,0,"""113806""",1,53.1,"""E8""","""Southampton""",1,0,1


8. You can test the accuracy of that model with the initial testing_table as follows:

In [8]:
testing_table = encoder.transform(testing_table)

test_tabular_dataset = testing_table.to_tabular_dataset("survived", extra_names=extra_names)
fitted_model.accuracy(test_tabular_dataset)


0.7614503816793893